In [1]:
import torch as t
from torch import nn
from torch.autograd import Variable
from torch.optim import Adam
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import CIFAR10, ImageFolder
import numpy as np
from torch  import optim
import torchvision.utils as vutil
from tensorboard_logger import Logger
import torchvision as tv
from torch.utils.data.dataloader import default_collate

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

from tensorboard_logger import configure, log_value, Logger


In [2]:
class Config:
    lr=0.0002
    nz=100# 噪声维度
    image_size=64
    image_size2=64
    nc=3# 图片三通道
    ngf=64 #生成图片
    ndf=64 #判别图片
    gpuids=2 #gpu
    beta1=0.5
    batch_size=256
    max_epoch=12# =1 when debug
    workers=4
    
opt=Config()

In [3]:
class MyImageFolder(ImageFolder):
    __init__ = ImageFolder.__init__
    def __getitem__(self, index):
        try:
            return super(MyImageFolder, self).__getitem__(index)
        except Exception as e:
            if index>0:index=index -1
            else: index = index+1
            return self.__getitem__(index)

# 训练数据集
dataset= MyImageFolder('/home/x/data/pre/train_new/nouse2/',
                transform=transforms.Compose([transforms.Scale(opt.image_size),
                                              transforms.RandomCrop(opt.image_size) ,
                                              transforms.ToTensor(),
                                              transforms.Normalize([0.5]*3,[0.5]*3)
                                             ]))
dataloader=t.utils.data.DataLoader(dataset,opt.batch_size,True,num_workers=opt.workers)


# 验证数据集
val_dataset=MyImageFolder('/home/x/data/pre/test_new/nouse',
                transform=transforms.Compose(
                                             [transforms.Scale(opt.image_size),
                                              transforms.RandomCrop(opt.image_size) ,
                                              transforms.ToTensor(),
                                              transforms.Normalize([0.5]*3,[0.5]*3)
                                             ]))

val_dataloader=t.utils.data.DataLoader(val_dataset, opt.batch_size, True, num_workers = opt.workers)

In [169]:
# model from GAN

class ModelD(nn.Module):
    def __init__(self,ngpu=None):
        super(ModelD,self).__init__()
        self.ngpu=ngpu
        self.model=nn.Sequential()
        self.model.add_module('conv1',nn.Conv2d(opt.nc,opt.ndf,4,2,1,bias=False))
        self.model.add_module('relu1',nn.LeakyReLU(0.2,inplace=True))
        
        self.model.add_module('conv2',nn.Conv2d(opt.ndf,opt.ndf*2,4,2,1,bias=False))
        self.model.add_module('bnorm2',nn.BatchNorm2d(opt.ndf*2))
        self.model.add_module('relu2',nn.LeakyReLU(0.2,inplace=True))
        
        self.model.add_module('conv3',nn.Conv2d(opt.ndf*2,opt.ndf*4,4,2,1,bias=False))
        self.model.add_module('bnorm3',nn.BatchNorm2d(opt.ndf*4))
        self.model.add_module('relu3',nn.LeakyReLU(0.2,inplace=True))
        
        self.model.add_module('conv4',nn.Conv2d(opt.ndf*4,opt.ndf*8,4,2,1,bias=False))
        self.model.add_module('bnorm4',nn.BatchNorm2d(opt.ndf*8))
        self.model.add_module('relu4',nn.LeakyReLU(0.2,inplace=True))
        
        self.model.add_module('conv5',nn.Conv2d(opt.ndf*8,1,4,1,0,bias=False))
#         self.model.add_module('sigmoid',nn.Sigmoid())
    def forward(self,input):
        gpuids=None
        if self.ngpu:
            gpuids=range(self.ngpu)
        return nn.parallel.data_parallel(self.model,input, device_ids=gpuids)


# define my classify model
class FinetuneModel(nn.Module):
    def __init__(self, pretrained_model, ngpu=opt.gpuids):
        self.ngpu = ngpu
        super(FinetuneModel,self).__init__()
        self.features = pretrained_model
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(512 * 4 * 4, 2048),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(2048, 2048),
            nn.ReLU(True),
            nn.Linear(2048, 2))      
        
    def forward(self, x):
        gpuids = None
        if self.ngpu:
            gpuids = range(self.ngpu)
        features =  self.features(x).view(-1,512*4*4)#, x, device_ids=gpuids,ou)
        return self.classifier(features)
       # return nn.parallel.data_parallel(self.classifier, features, device_ids=gpuids)

## tools to use for validate
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

        
        
def accuracy(output, target, topk=(1,)):
    """
    Computes the precision@k for the specified values of k
    @param output: score
    @param target: label
    
    """
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

def val(model): 
    model.eval()
    acc_meter = AverageMeter()
    loss_meter = AverageMeter()
    for ii,data in enumerate(val_dataloader):
        input,label=data 
        val_input = Variable(input, volatile = True).cuda()
        val_label = Variable(label.type(t.LongTensor), volatile = True).cuda()
        score = model(val_input)
        acc = accuracy(score.cpu().data,label)[0][0]
        val_loss = t.nn.functional.cross_entropy(score,val_label)
        n = input.size()[0]
        acc_meter.update(acc,n)
        loss_meter.update(val_loss.data,n)
        if acc_meter.count>1000:break # val only 3000pictures one time
    model.train()
    return acc_meter, loss_meter


In [170]:
netd=ModelD(opt.gpuids)
if hasattr(netd.model,'conv5'):del netd.model.conv5
model = FinetuneModel(netd)

In [50]:
model.load_state_dict(t.load('/home/cy/class_current_best_model_94.4813829787.nph'))

In [172]:

# def weight_init(m):
#     #模型参数初始化．　可以优化成为xavier 初始化
#     class_name=m.__class__.__name__
#     if class_name.find('Conv')!=-1:
        
#         print 'Conv',class_name
#         #xavier_normal(m.weight.data)
#         print (m.weight.data.mean(),t.var(m.weight.data))
#         m.weight.data.normal_(m.weight.data.mean(),t.var(m.weight.data)*10)
# #         m.weight.data.add_(t.randn(*(m.weight.size())).cuda() * 0.005)
# #         m.bias.data.zero_()
#         m.weight.data.normal_(0,0)
#     if class_name.find('Norm')!=-1:
#         #xavier_normal(m.weight.data)
#         print 'Norm',class_name
#         print (m.weight.data.mean(),t.var(m.weight.data))
#         m.weight.data.normal_(m.weight.data.mean(),t.var(m.weight.data)*10)
#         m.bias.data.zero_()
#     elif class_name.find('Linear')!=-1:
#         xavier_normal(m.weight.data)
#         m.bias.data.zero_()
#         print 'a',class_name
        #m.weight.data.normal_(0,0.02)
def weight_init(m):
    # 参数初始化。 可以改成xavier初始化方法
    class_name=m.__class__.__name__
    if class_name.find('Conv')!=-1: 
        m.weight.data.normal_(0,0.02)
    if class_name.find('Norm')!=-1:
        m.weight.data.normal_(1.0,0.02)
        m.bias.data.fill_(0)
    elif class_name.find('Linear')!=-1:
        xavier_normal(m.weight.data)
        m.bias.data.zero_()
        print 'a',class_name
        #m.weight.data.normal_(0,0.02)
model.apply(weight_init)

a Linear
a Linear
a Linear


FinetuneModel (
  (features): ModelD (
    (model): Sequential (
      (conv1): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (relu1): LeakyReLU (0.2, inplace)
      (conv2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (bnorm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
      (relu2): LeakyReLU (0.2, inplace)
      (conv3): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (bnorm3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
      (relu3): LeakyReLU (0.2, inplace)
      (conv4): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (bnorm4): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
      (relu4): LeakyReLU (0.2, inplace)
    )
  )
  (classifier): Sequential (
    (0): Dropout (p = 0.5)
    (1): Linear (8192 -> 2048)
    (2): ReLU (inplace)
    (3): Dropout (p = 0.5)
    (4): Linear (2048 -> 2048)
    (5)

In [174]:
optimizerFC=optim.Adam([{'params':model.features.parameters(),'lr':opt.lr},
                        {'params':model.classifier.parameters(),'lr':opt.lr}
                       ], betas = (opt.beta1, 0.999))

input=Variable(t.FloatTensor(opt.batch_size,opt.nc,opt.image_size,opt.image_size2))#.cuda()
label=Variable(t.LongTensor(opt.batch_size))#.cuda()
criterion = nn.CrossEntropyLoss()

if opt.gpuids:
    input = input.cuda()
    label = label.cuda()
    model.cuda()
    criterion.cuda()
step = 0

In [ ]:
optimizerFC=optim.Adam([{'params':model.features.parameters(),'lr':0.1*opt.lr},
                        {'params':model.classifier.parameters(),'lr':0.1*opt.lr}
                       ], betas = (opt.beta1, 0.999))
test_logger = Logger('../tensorboard_log/runs/cnn_classifier6')
max_acc = 95
for epoch in xrange(1,100):
    for ii, data in enumerate(dataloader,0):
        try:
            model.zero_grad()
            real,label_=data
            input.data.resize_(real.size()).copy_(real)
            label.data.resize_(input.size()[0]).copy_(label_)
            output=model(input)
            error_real=criterion(output,label)
            error_real.backward()
            D_x=output.data.mean()
            optimizerFC.step()
            
            step += 1
            if step>10:
                test_logger.log_value('error_d',error_real.data[0],step)
                test_logger.log_value('D_x',D_x,step)
            if ii%10==0 and ii>0:
                    a, l = val(model)
                    if a.avg >max_acc: 
                        max_acc=a.avg
                        t.save(model.state_dict(),'class_current_best_model_%s.nph' %max_acc)
                    test_logger.log_value('val_loss',l.avg[0],step)
                    test_logger.log_value('val_acc',a.avg,step)
                    test_logger.log_value('epoch', epoch)
                    test_logger.log_value('ii', ii)
        except Exception as e:
            print e
                 

/usr/local/lib/python2.7/dist-packages/PIL/Image.py:874: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')
/usr/local/lib/python2.7/dist-packages/PIL/Image.py:874: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')
/usr/local/lib/python2.7/dist-packages/PIL/Image.py:874: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')
/usr/local/lib/python2.7/dist-packages/PIL/Image.py:874: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')
/usr/local/lib/python2.7/dist-packages/PIL/Image.py:874: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')
/usr/local/lib/python2.7/dist-packages/PIL/Image.py:874: UserWarning: Palette images with Transparency   

In [163]:
# data=iter(dataloader)
optimizerFC=optim.Adam([{'params':model.features.parameters(),'lr':opt.lr},
                        {'params':model.classifier.parameters(),'lr':opt.lr}
                       ], betas = (opt.beta1, 0.999))
if True:
            model.zero_grad()
            real,label_=data.next()
            input.data.resize_(real.size()).copy_(real)
            label.data.resize_(input.size()[0]).copy_(label_)
            output=model(input)
            error_real=criterion(output,label)
            #error_real.backward()
            D_x=output.data.mean()
#             optimizerFC.step()
            
            step += 1


In [173]:
norm=model._modules.get('features')._modules['model']._modules['bnorm2']
norm.state_dict()

OrderedDict([('weight', 
               1.0160
               0.9839
               1.0094
               0.9934
               0.9920
               1.0041
               0.9975
               0.9501
               1.0135
               1.0041
               0.9802
               1.0213
               1.0394
               1.0022
               0.9972
               0.9704
               0.9905
               1.0094
               0.9948
               1.0138
               0.9808
               1.0118
               0.9922
               1.0142
               0.9913
               1.0277
               0.9796
               0.9981
               1.0128
               0.9911
               0.9674
               0.9841
               1.0043
               0.9747
               1.0377
               1.0220
               1.0052
               0.9788
               1.0000
               0.9728
               0.9983
               0.9861
               0.9953
               0.9728
       

In [109]:
for (ii,jj) in  (model._modules.get('features')._modules['model']._modules).iteritems():print ii,jj

conv1 Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
relu1 LeakyReLU (0.2, inplace)
conv2 Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
bnorm2 BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
relu2 LeakyReLU (0.2, inplace)
conv3 Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
bnorm3 BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
relu3 LeakyReLU (0.2, inplace)
conv4 Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
bnorm4 BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
relu4 LeakyReLU (0.2, inplace)


In [133]:
tmp = t.Tensor(128,128,16,16)
def fn(x,y,z):tmp.copy_(z.data)
r=model._modules.get('features')._modules['model']._modules['conv2'].register_forward_hook(fn)

In [125]:
r={
ii:mm.register_forward_hook(fn)
 for ii,mm in model._modules.get('features')._modules['model']._modules.iteritems()
}

In [127]:

for ii in list(model.parameters()):
    print ii

Parameter containing:
(0 ,0 ,.,.) = 
1.00000e-02 *
 -2.9443  1.2740 -1.3675 -3.5839
 -1.4194 -1.4231  1.3433  1.3199
  1.1498  3.7366  2.0612 -4.7754
  2.5386 -1.8528  0.9180  0.9751

(0 ,1 ,.,.) = 
1.00000e-02 *
  0.4435  2.6998  0.4831  2.4340
 -0.8711  0.9448 -1.3786  4.0447
 -0.8067 -0.4061  1.3563  4.6852
 -3.5009  0.6244 -4.9069 -0.6132

(0 ,2 ,.,.) = 
1.00000e-02 *
  1.2218  1.3931 -0.5838  1.0902
 -1.0052 -1.2321 -2.3953 -1.1548
  2.6572  1.1532 -2.8928 -1.7039
  0.4397  1.5250  2.4286  1.6358
     ⋮ 

(1 ,0 ,.,.) = 
1.00000e-02 *
  0.6301 -0.4508 -2.5147  3.0496
 -1.3163  0.8824 -1.0645 -2.8138
  1.0868  1.8104  1.0773  2.4678
  1.2768 -0.9345 -2.3650  1.8713

(1 ,1 ,.,.) = 
1.00000e-02 *
 -0.4537  0.2801  1.9432 -0.9368
 -2.9854  0.7687  2.7918 -0.6559
 -2.3967 -0.2449  2.4610 -1.9675
 -1.6246  0.7765 -0.4811 -0.5398

(1 ,2 ,.,.) = 
1.00000e-02 *
  2.6259  0.8802 -0.8576  1.7116
  2.6498 -1.3863  0.3775  2.0457
 -1.7439  1.2663  1.2666  4.0195
 -0.4869 -0.8226  2.0717  3.8125

In [38]:
r=list(model.children())[0]

In [47]:
rr=list(r.children())[0]

In [51]:
list(rr.modules())[1]

Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)

In [57]:
output

Variable containing:
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.1884 -0.1491
 0.

In [71]:
for ii in model.parameters():
    print ii.size()

torch.Size([64, 3, 4, 4])
torch.Size([128, 64, 4, 4])
torch.Size([128])
torch.Size([128])
torch.Size([256, 128, 4, 4])
torch.Size([256])
torch.Size([256])
torch.Size([512, 256, 4, 4])
torch.Size([512])
torch.Size([512])
torch.Size([2048, 8192])
torch.Size([2048])
torch.Size([2048, 2048])
torch.Size([2048])
torch.Size([2, 2048])
torch.Size([2])


In [139]:
norm=model._modules.get('features')._modules['model']._modules['bnorm2']

In [149]:
norm(Variable(t.ones(128,128,16,16)).cuda())

Variable containing:
( 0 , 0 ,.,.) = 
 nan nan nan  ...  nan nan nan
 nan nan nan  ...  nan nan nan
 nan nan nan  ...  nan nan nan
     ...       ⋱       ...    
 nan nan nan  ...  nan nan nan
 nan nan nan  ...  nan nan nan
 nan nan nan  ...  nan nan nan

( 0 , 1 ,.,.) = 
 nan nan nan  ...  nan nan nan
 nan nan nan  ...  nan nan nan
 nan nan nan  ...  nan nan nan
     ...       ⋱       ...    
 nan nan nan  ...  nan nan nan
 nan nan nan  ...  nan nan nan
 nan nan nan  ...  nan nan nan

( 0 , 2 ,.,.) = 
 nan nan nan  ...  nan nan nan
 nan nan nan  ...  nan nan nan
 nan nan nan  ...  nan nan nan
     ...       ⋱       ...    
 nan nan nan  ...  nan nan nan
 nan nan nan  ...  nan nan nan
 nan nan nan  ...  nan nan nan
    ... 

( 0 ,125,.,.) = 
 nan nan nan  ...  nan nan nan
 nan nan nan  ...  nan nan nan
 nan nan nan  ...  nan nan nan
     ...       ⋱       ...    
 nan nan nan  ...  nan nan nan
 nan nan nan  ...  nan nan nan
 nan nan nan  ...  nan nan nan

( 0 ,126,.,.) = 
 nan nan nan 

In [161]:
r.remove()

In [159]:
norm.state_dict()

OrderedDict([('weight', 
               1.0116
               1.0103
               1.0261
               1.0188
               1.0030
               1.0272
               1.0045
               1.0113
               1.0468
               1.0065
               1.0120
               1.0227
               1.0256
               1.0049
               0.9773
               1.0045
               1.0165
               1.0153
               0.9943
               1.0054
               0.9903
               0.9961
               1.0029
               1.0181
               1.0102
               0.9919
               1.0241
               1.0282
               1.0351
               0.9941
               1.0222
               1.0032
               0.9999
               1.0166
               1.0179
               0.9954
               1.0020
               1.0060
               0.9920
               0.9797
               0.9986
               1.0179
               0.9915
               1.0130
       